In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.decomposition import PCA
import os

In [2]:
d_parser = lambda x : datetime.strptime(x,"%d.%m.%Y %H:%M:%S.%f")

In [3]:
df = pd.read_csv('E40_Bhilai_with_signalName.txt', sep=';')
df

,Time,[0:0],[0:1],[0:2],[0:3],[0:4],[0:6],[0:7],[0:8],[0:10],...,[0.199],[0.200],[0.201],[0.202],[0.203],[0.204],[0.205],[0.206],[0.207],[0.208]
0,time,DIV_SH2_Act_Position (PID),DSH2 Trigger Point for diverter,DIV_SH@-SpeedRef (Drive Unit),DIV_SH@-SpeedFEEDBACK (Drive Unit),DIV_SH2_Act_Position from encoder,DSH2_Theor_Position,DSH2_Drive_Speed_Feedback,DSH2_Theor_Speed,DIV_SH2_SET_Position (PID),...,SH2_WORK_DB.AUTOCHOP_RequestOFF,Chopping request Memory ON when billet at shear,Chopping request for minimum inter billet,Manual Chopping request Summary,Bar Presence at shear knives cross and out of ...,CCL Almost Full,CCL Full,LFA Automatic,SH2_Calibrated speed < minimum,SH2_Calibrated speed > maximum


In [4]:
df.iloc[0].values

array(['time', 'DIV_SH2_Act_Position (PID)',
       'DSH2 Trigger Point for diverter', 'DIV_SH@-SpeedRef (Drive Unit)',
       'DIV_SH@-SpeedFEEDBACK (Drive Unit)',
       'DIV_SH2_Act_Position from encoder', 'DSH2_Theor_Position',
       'DSH2_Drive_Speed_Feedback', 'DSH2_Theor_Speed',
       'DIV_SH2_SET_Position (PID)', 'DIVSH2_Point_Curve',
       'SH1 Head Cut off Length Accum', 'DSH2_Theor_Speed(Input for PID)',
       'DSH2_Theor_Speed(Out  PID)', 'SH1 Head Cut off Length',
       'SH1_HeadPosition', 'SH1_TailPosition', 'SH1_Act_Position(forPID)',
       'SH1_Teor_Position (for PID)', 'SH2_HeadPosition',
       'SH2_TailPosition', 'SH2_Act_Position(forPID)',
       'SH2_Teor_Position (for PID)', 'SH3_HeadPosition',
       'SH3_TailPosition', 'SH3_NEW_HeadPosition',
       'SH3_Act_Position(forPID)', 'SH3_Teor_Position (for PID)',
       'SH3_Produced bar length', 'OPT_RemLenght_before_SH1',
       'OPT_RemLenght_before_SH2', 'OPT_RemLenght_before_SH3',
       'OPT_RemLenght_befo

In [5]:
df.set_index('Time',inplace = True)

In [6]:
imp_signal = pd.read_excel('IMPORTANT SIGNALS.xlsx')
imp_signal

,OVERSHOOT,Unnamed: 1
0,DCVAH New Head Pos L1,[3:19]
1,DCVAH Remain Length before CVAH L1,[3:21]
2,Stand #2 downstream HMD material detection sig...,[9.11]
3,Stand #4 downstream HMD material detection sig...,[9.12]
4,Stand #6 downstream HMD material detection sig...,[9.13]
...,...,...
400,Stand #8 downstream HMD material detection sig...,[9.15]
401,Stand #7 upstream HMD material detection signa...,[9.14]
402,Stand #6 downstream HMD material detection sig...,[9.13]
403,Stand #4 downstream HMD material detection sig...,[9.12]


In [7]:
imp_signal.columns

Index(['                                                OVERSHOOT ', 'Unnamed: 1'], dtype='object')

In [8]:
imp_signal.dropna(axis=0,inplace= True)
imp_signal

,OVERSHOOT,Unnamed: 1
0,DCVAH New Head Pos L1,[3:19]
1,DCVAH Remain Length before CVAH L1,[3:21]
2,Stand #2 downstream HMD material detection sig...,[9.11]
3,Stand #4 downstream HMD material detection sig...,[9.12]
4,Stand #6 downstream HMD material detection sig...,[9.13]
...,...,...
400,Stand #8 downstream HMD material detection sig...,[9.15]
401,Stand #7 upstream HMD material detection signa...,[9.14]
402,Stand #6 downstream HMD material detection sig...,[9.13]
403,Stand #4 downstream HMD material detection sig...,[9.12]


In [9]:
y = []

for x in imp_signal['                                                OVERSHOOT '].values:
    if x not in y:
        y.append(x)
        print(x)
y

DCVAH New Head Pos L1
DCVAH Remain Length before CVAH L1
Stand #2 downstream HMD material detection signal (=JD21A02DVM-B011)
Stand #4 downstream HMD material detection signal (=JD21A04DVM-B011)
Stand #6 downstream HMD material detection signal (=JD21A06DVM-B011)
Stand #7 upstream HMD material detection signal (=JD21B01CVS-B011)
Stand #8 downstream HMD material detection signal (=JD31A08DVM-B011)
Stand #9 downstream HMD material detection signal (=JD31A09FAV-B001)
Stand #10 downstream HMD material detection signal (=JD31A10FAV-B001)
Stand #11 downstream HMD material detection signal (=JD31A11FAV-B001)
Stand #12 downstream HMD material detection signal (=JD31B01CVS-B011)
SH1_Act_Position(forPID)
SH2_Act_Position(forPID)
DCVR L2 Act Position
TWC Line 2 cobble detection HMD
TWC Line 1 cobble detection HMD
SH2_WORK_DB.AUTOCHOP_RequestOFF
E10PLC01 Mat.det. HMD31 FFB exit L2
E36PLC01 Mat.det. HMD37 CVRD entry L2
E36PLC01 Mat.det. HMD38 TWC2A entry
E36PLC01 Mat.det. HMD39 TWC2B entry
Speed fr

['DCVAH New Head Pos L1',
 'DCVAH Remain Length before CVAH L1',
 'Stand #2 downstream HMD material detection signal (=JD21A02DVM-B011)',
 'Stand #4 downstream HMD material detection signal (=JD21A04DVM-B011)',
 'Stand #6 downstream HMD material detection signal (=JD21A06DVM-B011)',
 'Stand #7 upstream HMD material detection signal (=JD21B01CVS-B011)',
 'Stand #8 downstream HMD material detection signal (=JD31A08DVM-B011)',
 'Stand #9 downstream HMD material detection signal (=JD31A09FAV-B001)',
 'Stand #10 downstream HMD material detection signal (=JD31A10FAV-B001)',
 'Stand #11 downstream HMD material detection signal (=JD31A11FAV-B001)',
 'Stand #12 downstream HMD material detection signal (=JD31B01CVS-B011)',
 'SH1_Act_Position(forPID)',
 'SH2_Act_Position(forPID)',
 'DCVR L2 Act Position',
 'TWC Line 2 cobble detection HMD',
 'TWC Line 1 cobble detection HMD',
 'SH2_WORK_DB.AUTOCHOP_RequestOFF',
 'E10PLC01 Mat.det. HMD31 FFB exit L2',
 'E36PLC01 Mat.det. HMD37 CVRD entry L2',
 'E3

In [10]:
len(y)

117

In [11]:
z=[]
for x in df.iloc[0].values:
    if x in y:
        z.append(x)
z

['SH1_Act_Position(forPID)',
 'SH2_Act_Position(forPID)',
 'SH1_CHOP_Pushbutton_MainPulpit',
 'SH2_CHOP_Pushbutton_MainPulpit',
 'SH2_WORK_DB.AUTOCHOP_RequestOFF']

In [12]:
df1 = pd.read_csv('BSP/new_oct_5/00.28.14_01.28.14_oct.txt',sep='\t')
df1

,Time,[0:0],[0:1],[0:2],[0:3],[0:4],[0:6],[0:7],[0:8],[0:10],...,[7.233],[7.234],[7.235],[7.236],[7.237],[7.238],[7.239],[7.240],[7.241],[7.243]
0,05.10.2021 00:28:14.460000,-173.984,3689.29,0,0.29,4.0,1150,0.11,0,115,...,0,0,0,0,0,0,0,0,0,0
1,05.10.2021 00:28:15.460000,-173.984,3689.29,0,-0.11,4.0,1150,-0.01,0,115,...,0,0,0,0,0,0,0,0,0,0
2,05.10.2021 00:28:16.460000,-173.984,3689.29,0,-0.54,4.0,1150,-0.16,0,115,...,0,0,0,0,0,0,0,0,0,0
3,05.10.2021 00:28:17.460000,-173.984,3689.29,0,-1.00,4.0,1150,-0.07,0,115,...,0,0,0,0,0,0,0,0,0,0
4,05.10.2021 00:28:18.460000,-173.984,3689.29,0,0.03,4.0,1150,0.01,0,115,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,05.10.2021 01:28:09.460000,-173.984,3689.29,0,0.48,4.0,1150,0.08,0,115,...,0,0,0,0,0,0,0,0,0,0
3596,05.10.2021 01:28:10.460000,-173.984,3689.29,0,0.17,4.0,1150,0.00,0,115,...,0,0,0,0,0,0,0,0,0,0
3597,05.10.2021 01:28:11.460000,-173.984,3689.29,0,0.25,4.0,1150,0.08,0,115,...,0,0,0,0,0,0,0,0,0,0
3598,05.10.2021 01:28:12.460000,-173.984,3689.29,0,0.38,4.0,1150,0.11,0,115,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df2 = pd.read_csv('signals.txt',sep=';',header = None)
df2

,0,1,2,3,4,5,6,7,8,9,...,1584,1585,1586,1587,1588,1589,1590,1591,1592,1593
0,[0:0],[0:1],[0:2],[0:3],[0:4],[0:6],[0:7],[0:8],[0:10],[0:12],...,[7.233],[7.234],[7.235],[7.236],[7.237],[7.238],[7.239],[7.240],[7.241],[7.243]
1,DIV_SH2_Act_Position (PID),DSH2 Trigger Point for diverter,DIV_SH@-SpeedRef (Drive Unit),DIV_SH@-SpeedFEEDBACK (Drive Unit),DIV_SH2_Act_Position from encoder,DSH2_Theor_Position,DSH2_Drive_Speed_Feedback,DSH2_Theor_Speed,DIV_SH2_SET_Position (PID),DIVSH2_Point_Curve,...,JD01E20_COMM_DB.CPU1_MPI_FROM.CVR_CHOP_REQ,HMI_ALARMS_DB.PLC_BA_PLC_NET_FLT09,Chopping request for TAIL OPT overshoot,DCVRD_WORK_DB.ChpReqOther_TAIL OPT overshoot,DCVRD_WORK_DB.ChpReqMemory,CVR_PH01_NoViewAlarmONS,CVR_PH01_NoViewAlarm,CVR_PH01_BLK,DCVR_WORK_DB.DriveCommFlt,ShortPieceforHMD37
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
x = []

for i in y:
    if i in df2.iloc[1].values:
        x.append(i)
x

['DCVAH New Head Pos L1',
 'DCVAH Remain Length before CVAH L1',
 'Stand #2 downstream HMD material detection signal (=JD21A02DVM-B011)',
 'Stand #4 downstream HMD material detection signal (=JD21A04DVM-B011)',
 'Stand #6 downstream HMD material detection signal (=JD21A06DVM-B011)',
 'Stand #7 upstream HMD material detection signal (=JD21B01CVS-B011)',
 'Stand #8 downstream HMD material detection signal (=JD31A08DVM-B011)',
 'Stand #9 downstream HMD material detection signal (=JD31A09FAV-B001)',
 'Stand #10 downstream HMD material detection signal (=JD31A10FAV-B001)',
 'Stand #11 downstream HMD material detection signal (=JD31A11FAV-B001)',
 'Stand #12 downstream HMD material detection signal (=JD31B01CVS-B011)',
 'SH1_Act_Position(forPID)',
 'SH2_Act_Position(forPID)',
 'DCVR L2 Act Position',
 'TWC Line 2 cobble detection HMD',
 'TWC Line 1 cobble detection HMD',
 'SH2_WORK_DB.AUTOCHOP_RequestOFF',
 'E10PLC01 Mat.det. HMD31 FFB exit L2',
 'E36PLC01 Mat.det. HMD37 CVRD entry L2',
 'E3

In [19]:
len(x)

114

In [20]:
z = []

for i in y:
    if i not in x:
        z.append(i)
z

['Speed from SpeedMeter L2',
 'Looper FFB L2 HMD 27 Status',
 'E10PLC01 Mat.det. HMD31 FFB entry L2']

In [21]:
df2.columns = df2.iloc[1]
df2

1,DIV_SH2_Act_Position (PID),DSH2 Trigger Point for diverter,DIV_SH@-SpeedRef (Drive Unit),DIV_SH@-SpeedFEEDBACK (Drive Unit),DIV_SH2_Act_Position from encoder,DSH2_Theor_Position,DSH2_Drive_Speed_Feedback,DSH2_Theor_Speed,DIV_SH2_SET_Position (PID),DIVSH2_Point_Curve,...,JD01E20_COMM_DB.CPU1_MPI_FROM.CVR_CHOP_REQ,HMI_ALARMS_DB.PLC_BA_PLC_NET_FLT09,Chopping request for TAIL OPT overshoot,DCVRD_WORK_DB.ChpReqOther_TAIL OPT overshoot,DCVRD_WORK_DB.ChpReqMemory,CVR_PH01_NoViewAlarmONS,CVR_PH01_NoViewAlarm,CVR_PH01_BLK,DCVR_WORK_DB.DriveCommFlt,ShortPieceforHMD37
0,[0:0],[0:1],[0:2],[0:3],[0:4],[0:6],[0:7],[0:8],[0:10],[0:12],...,[7.233],[7.234],[7.235],[7.236],[7.237],[7.238],[7.239],[7.240],[7.241],[7.243]
1,DIV_SH2_Act_Position (PID),DSH2 Trigger Point for diverter,DIV_SH@-SpeedRef (Drive Unit),DIV_SH@-SpeedFEEDBACK (Drive Unit),DIV_SH2_Act_Position from encoder,DSH2_Theor_Position,DSH2_Drive_Speed_Feedback,DSH2_Theor_Speed,DIV_SH2_SET_Position (PID),DIVSH2_Point_Curve,...,JD01E20_COMM_DB.CPU1_MPI_FROM.CVR_CHOP_REQ,HMI_ALARMS_DB.PLC_BA_PLC_NET_FLT09,Chopping request for TAIL OPT overshoot,DCVRD_WORK_DB.ChpReqOther_TAIL OPT overshoot,DCVRD_WORK_DB.ChpReqMemory,CVR_PH01_NoViewAlarmONS,CVR_PH01_NoViewAlarm,CVR_PH01_BLK,DCVR_WORK_DB.DriveCommFlt,ShortPieceforHMD37
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
for i in df.iloc[0].values:
    if i not in x:
        x.append(i)
x

['DCVAH New Head Pos L1',
 'DCVAH Remain Length before CVAH L1',
 'Stand #2 downstream HMD material detection signal (=JD21A02DVM-B011)',
 'Stand #4 downstream HMD material detection signal (=JD21A04DVM-B011)',
 'Stand #6 downstream HMD material detection signal (=JD21A06DVM-B011)',
 'Stand #7 upstream HMD material detection signal (=JD21B01CVS-B011)',
 'Stand #8 downstream HMD material detection signal (=JD31A08DVM-B011)',
 'Stand #9 downstream HMD material detection signal (=JD31A09FAV-B001)',
 'Stand #10 downstream HMD material detection signal (=JD31A10FAV-B001)',
 'Stand #11 downstream HMD material detection signal (=JD31A11FAV-B001)',
 'Stand #12 downstream HMD material detection signal (=JD31B01CVS-B011)',
 'SH1_Act_Position(forPID)',
 'SH2_Act_Position(forPID)',
 'DCVR L2 Act Position',
 'TWC Line 2 cobble detection HMD',
 'TWC Line 1 cobble detection HMD',
 'SH2_WORK_DB.AUTOCHOP_RequestOFF',
 'E10PLC01 Mat.det. HMD31 FFB exit L2',
 'E36PLC01 Mat.det. HMD37 CVRD entry L2',
 'E3

In [24]:
len(x)

374

In [30]:
df2[x]

1,DCVAH New Head Pos L1,DCVAH Remain Length before CVAH L1,Stand #2 downstream HMD material detection signal (=JD21A02DVM-B011),Stand #4 downstream HMD material detection signal (=JD21A04DVM-B011),Stand #6 downstream HMD material detection signal (=JD21A06DVM-B011),Stand #7 upstream HMD material detection signal (=JD21B01CVS-B011),Stand #8 downstream HMD material detection signal (=JD31A08DVM-B011),Stand #9 downstream HMD material detection signal (=JD31A09FAV-B001),Stand #10 downstream HMD material detection signal (=JD31A10FAV-B001),Stand #11 downstream HMD material detection signal (=JD31A11FAV-B001),...,SH2 Exit Bottom Open Closed,Chopping request Memory ON when billet at shear,Chopping request for minimum inter billet,Manual Chopping request Summary,Bar Presence at shear knives cross and out of shear,CCL Almost Full,CCL Full,LFA Automatic,SH2_Calibrated speed < minimum,SH2_Calibrated speed > maximum
0,[5:13],[5:15],[1.3],[1.4],[1.5],[1.6],[1.7],[1.8],[1.9],[1.10],...,[0.198],[0.200],[0.201],[0.202],[0.203],[0.204],[0.205],[0.206],[0.207],[0.208]
1,DCVAH New Head Pos L1,DCVAH Remain Length before CVAH L1,Stand #2 downstream HMD material detection sig...,Stand #4 downstream HMD material detection sig...,Stand #6 downstream HMD material detection sig...,Stand #7 upstream HMD material detection signa...,Stand #8 downstream HMD material detection sig...,Stand #9 downstream HMD material detection sig...,Stand #10 downstream HMD material detection si...,Stand #11 downstream HMD material detection si...,...,SH2 Exit Bottom Open Closed,Chopping request Memory ON when billet at shear,Chopping request for minimum inter billet,Manual Chopping request Summary,Bar Presence at shear knives cross and out of ...,CCL Almost Full,CCL Full,LFA Automatic,SH2_Calibrated speed < minimum,SH2_Calibrated speed > maximum
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
imp_signalAnd_E40 = df2[x]

In [28]:
imp_signalAnd_E40.drop([1,2],inplace=True)

/home/suraj/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [29]:
imp_signalAnd_E40

1,DCVAH New Head Pos L1,DCVAH Remain Length before CVAH L1,Stand #2 downstream HMD material detection signal (=JD21A02DVM-B011),Stand #4 downstream HMD material detection signal (=JD21A04DVM-B011),Stand #6 downstream HMD material detection signal (=JD21A06DVM-B011),Stand #7 upstream HMD material detection signal (=JD21B01CVS-B011),Stand #8 downstream HMD material detection signal (=JD31A08DVM-B011),Stand #9 downstream HMD material detection signal (=JD31A09FAV-B001),Stand #10 downstream HMD material detection signal (=JD31A10FAV-B001),Stand #11 downstream HMD material detection signal (=JD31A11FAV-B001),...,SH2 Exit Bottom Open Closed,Chopping request Memory ON when billet at shear,Chopping request for minimum inter billet,Manual Chopping request Summary,Bar Presence at shear knives cross and out of shear,CCL Almost Full,CCL Full,LFA Automatic,SH2_Calibrated speed < minimum,SH2_Calibrated speed > maximum
0,[5:13],[5:15],[1.3],[1.4],[1.5],[1.6],[1.7],[1.8],[1.9],[1.10],...,[0.198],[0.200],[0.201],[0.202],[0.203],[0.204],[0.205],[0.206],[0.207],[0.208]


In [31]:
imp_signalAnd_E40.csv')imp_signalAnd_E40.to_csv('